# Praca domowa 1 - odkodowanie zdań za pomocą LLM

## Wstęp

W pierwszej pracy domowej, naszym zadaniem jest odkodowanie zdań za pomocą wybranego LLM'a. Zdecydowałem się zrealizować wersję pracy domowej z lokalnie postawionym LLM'em - w tym celu użyję frameworka `ollama`, który znacznie ułatwia pobieranie open sourcowych LLM'ów, a przede wszystkim dostarcza API automatyzujące inference i wspierające prawie wszystkie najlepsze open sourcowe modele. Dzięki temu będziemy mogli niskim kosztem przetestować wiele LLM'ów.

Zacznijmy jednak od wczytania danych:

In [ ]:
file_path = 'homework_1_ang.txt'
sentences = []

with open(file_path, 'r') as file:
    sentences = file.readlines()

sentences = [sentence.strip() for sentence in sentences]
for sentence in sentences:
    print(sentence)


Uif qmbdfcp fggfdu jt b qifopnfopo jo xijdi qfpqmf fyqfsjfodf bo jnqspwfnfou jo ifbmui bgufs ubljoh b tvctubodf xjui op nfejdjobm qspqfsujft cfdbvtf uifz cfmjfwf ju xpslt.
Vjg jwocp dtckp ku pgwtqrncuvke, yjkej ogcpu kv ecp ejcpig cpf cfcrv vjtqwijqwv nkhg kp tgurqpug vq gzrgtkgpegu.
Wkh Plojudp hashulphqw vkrzhg wkdw shrsoh duh zloolqj wr rehb dxwkrulwb hyhq zkhq wkhlu dfwlrqv pdb kdup rwkhuv.
Xli fcwxerhiv ijjigx qeoiw tistpi piww pmoipc xs sjjiv lipt alir qsvi tistpi evi evsyrh—xlic ewwyqi xlex wsqisri ipwi ampp hs mx.
Ujtuqj wjrjrgjw ymj gjlnssnsl fsi jsi tk f qnxy tk nsktwrfynts gjyyjw, bmnhm nx pstbs fx ymj uwnrfhd jkkjhy fsi ymj wjhjshd jkkjhy.
Vxurutmkj yzxkyy igt rkgj zu nkgrzn vxuhrksy yain gy nkgxz joykgyk, g ckgqktkj ossatk yeyzks, gtj jkvxkyyout.
Ltvapvuz pumsblujl jvnupapcl wyvjlzzlz zbjo hz tltvyf huk kljpzpvu-thrpun—zayvun zaylzz jhu opukly svnpjhs aopurpun.
Pweizl Oizlvmz'a bpmwzg wn uctbqxtm qvbmttqomvkma acoomaba bpib bpmzm izm dizqwca bgxma wn qvbmttqomvkm, ackp ia 

Wczytajmy `ollama` i przetestujmy działanie za pomocą jakiegoś losowego prompta.

In [11]:
import ollama
from ollama import chat
from ollama import ChatResponse


response: ChatResponse = chat(model='llama3.1:latest', messages=[
  {
    'role': 'user',
    'content': 'Dlaczego grupa WB2: Osobowość w sztucznej inteligencji jest fajna?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)


Oto kilka powodów, dlaczego grupa WB2: Osobowość w Sztucznej Inteligencji może być interesująca:

1. **Zrozumienie ludzkiej psychozy**: badania nad osobowością sztuczną mogą pomóc w lepszym zrozumieniu mechanizmów psychicznych i procesów myślowych, co może przynieść korzyści dla psychologii klinicznej.
2. **Rozwój AI bardziej ludzka**: celem tej grupy jest utworzenie modeli sztucznej inteligencji, które mogą symulować ludzką osobowość, co umożliwi stworzenie bardziej naturalnych i skutecznych systemów komunikacji.
3. **Innowacje technologiczne**: badania w tej dziedzinie mogą prowadzić do innowacji technologicznych, takich jak rozwój inteligentnych asystentów, systemów sztucznej inteligencji dla zdrowia psychicznego czy nawet tworzenie postaci wirtualnych z osobowością.
4. **Zastosowania praktyczne**: wyniki badań mogą być wykorzystywane w różnych dziedzinach, takich jak psychologia kliniczna, edukacja, komunikacja biznesowa czy media społecznościowe.
5. **Interdyscyplinarność**: badan

Wszystko wydaje się działać prawidłowo. W celu odkodowanie zdań, pobrałem najpierw lokalnie pięć modeli. Na początku będziemy używać defaultowych hiperparametrów dla modeli językowych i jak nie poradzimy sobie z odszyfrowaniem zdań to spróbujemy dokonać może jakichś zmian.

In [41]:
ollama.list()['models']


[Model(model='gemma3:27b', modified_at=datetime.datetime(2025, 3, 15, 19, 41, 0, 144545, tzinfo=TzInfo(+01:00)), digest='30ddded7fba6d6f9c2f26661e2feba2d7a26a75e20a817538c41c3716d92609d', size=17396936887, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='27.4B', quantization_level='Q4_K_M')),
 Model(model='deepseek-r1:14b', modified_at=datetime.datetime(2025, 3, 15, 19, 7, 45, 129206, tzinfo=TzInfo(+01:00)), digest='ea35dfe18182f635ee2b214ea30b7520fe1ada68da018f8b395b444b662d4f1a', size=8988112040, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='14.8B', quantization_level='Q4_K_M')),
 Model(model='llama3.1:latest', modified_at=datetime.datetime(2025, 3, 13, 22, 2, 49, 85909, tzinfo=TzInfo(+01:00)), digest='46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e', size=4920753328, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama

Zasługują one na mały opis:
- Llama 3.1 8B - wersja `3.1` modelu od Mety (na większe/nowsze nie pozwalały mi ograniczenia sprzętowe - 16 GM RAM)
- Falcon3 10B - model open source od Technology Innovation Institute w Abu Dhabi, swojego czasu najlepszy open sourcowy model na świecie, miał premierę już ponad rok temu, a w świecie LLM'ów to jak stulecie
- Mistral 7B - model od francuskiego startupu Mistral AI, jeden z najlepszych modeli open source
- Gemma3 12B - nowy multimodalny model od Google, miał premierę raptem kilka dni temu, więc może dawać bardzo ciekawe wyniki i super będzie go przetestować
- DeepSeek-R1 8B - chiński model od DeepSeek, który osiąga rezultaty porównywalne do OpenAI-o1 jeżeli chodzi o kodowanie, matematykę i rozumowanie


## Dekodowanie

W celu odkodowania zdań użyję najpierw najbardziej basicowego promptu:

In [17]:
sentences_combined = ' '.join(sentences)
all_models = ['llama3.1:latest', 'falcon3:10b', 'mistral:latest', 'gemma3:12b', 'deepseek-r1:8b']

responses = []
for model in all_models:
    response = chat(model=model, messages=[{'role': 'user', 'content': 'Decode this sentences: ' + sentences_combined}])
    responses.append(response)

for idx in range(len(responses)):
    print("-------------------------------")
    print(f"Model: {all_models[idx]}")
    print(responses[idx]['message']['content'])
    print("\n")


-------------------------------
Model: llama3.1:latest
A coded message!

After some analysis, I believe the text is encoded using a simple substitution cipher, where each letter has been replaced by another letter a fixed number of positions down the alphabet.

Specifically, it appears to be an Atbash cipher, where A becomes Z, B becomes Y, C becomes X, and so on. This type of cipher is often used for simplicity and can be easily broken if not implemented correctly.

Using this technique, I'll attempt to decode the message:

**Initial decoded text:**

a = n
b = m
c = l
d = k
e = j
f = i
g = h
h = g
i = f
j = e
k = d
l = c
m = b
n = a
o = z
p = y
q = x
r = w
s = v
t = u
u = t
v = s
w = r
x = q
y = p
z = o

**Decoded message:**

Nothing is ever where it appears to be, including the truth. (Anonymous)

A true friend is the greatest of all blessings, and that which gives the greatest amount of happiness. (Edgar F. Smith)

The Pundit has a very simple theory about life—wherever he goes, not

Słabe wyniki. Może to wynikać z architektury LLMów i tego jak działają transformery. Spróbujmy zdekodować zdania osobno. Zacznijmy od pierwszego zdania:


In [ ]:
sentences[0]


'Uif qmbdfcp fggfdu jt b qifopnfopo jo xijdi qfpqmf fyqfsjfodf bo jnqspwfnfou jo ifbmui bgufs ubljoh b tvctubodf xjui op nfejdjobm qspqfsujft cfdbvtf uifz cfmjfwf ju xpslt.'

In [22]:
responses = []
for idx in range(len(all_models)):
    response = chat(model=all_models[idx], messages=[{'role': 'user', 'content': 'Decode this sentence: ' + sentences[0]}])
    responses.append(response)
    print("-------------------------------")
    print(f"Model: {all_models[idx]}")
    print(responses[idx]['message']['content'])
    print("\n")


-------------------------------
Model: llama3.1:latest
This sentence was encoded using the Vigenère cipher, which is a polyalphabetic substitution cipher that uses a keyword to determine the substitution for each letter.

The decoded text is:

"You are a human being you have your place in the sun and you have no right not to expect others to know it."

Decoded by: The keyword used was 'humanity'.


-------------------------------
Model: falcon3:10b
### Decoded Sentence:
The quick brown fox jumps over the lazy dog when assessing the quality of its food in comparison to other types of sustenance.


-------------------------------
Model: mistral:latest
 The encoded sentence appears to be a Caesar cipher with a shift of 13. To decode it, we'll use the reverse process of a Caesar cipher. Here's how it looks:

U (V) if (if) qmbdfcp (xkzhwbav) fggfdu (hknlmpyw) jt (kt) b (b) qifopnfopo (pzifdgvfzd) jo (jr) xijdi (xyzdfghji) qfpqmf (lrswelpma) fyqfsjfodf (hvzyrdtyvof) bo (do) jnqspwfnfou (knao

Żaden model nie potrafił poprawnie odkodować nawet tego jednego zdania. Najbardziej jednak obiecujące były odpowiedzi Gamma3 oraz DeepSeek'a. To tych dwóch modeli użyjemy do dalszych analiz.

In [ ]:
best_models = ['gemma3:12b', 'deepseek-r1:8b']

responses = []
for idx in range(len(best_models)):
    response = chat(model=best_models[idx], messages=[{'role': 'user', 'content': 'Decode this sentence decoded using Caesar Cipher: ' + sentences[0]}])
    responses.append(response)
    print("-------------------------------")
    print(f"Model: {all_models[idx]}")
    print(responses[idx]['message']['content'])
    print("\n")


-------------------------------
Model: llama3.1:latest
This is a Caesar cipher! Let's try shifting the letters back one position. This is a common starting point for Caesar cipher decryption.

Here's the decoded sentence:

**The placebe effect is a sineopnmo in which people experience an improvement in health after taking a substitute with no legitimate sacrifices because they believe it works.**

It seems like there might be a slight error in the original encoding, as "sineopnmo" is not a standard word. It's likely meant to be "symptom."  If we assume that's the case, the sentence becomes:

**The placebo effect is a symptom in which people experience an improvement in health after taking a substitute with no legitimate sacrifices because they believe it works.**



Let me know if you'd like to try decoding another message!


-------------------------------
Model: falcon3:10b
<think>
Okay, so I have this ciphered sentence that I need to decode using the Caesar Cipher. The sentence is: 

Porażka, spróbujmy poeksperymentować trochę z prompt engineeringiem.

In [24]:
responses = []
for idx in range(len(best_models)):
    response = chat(model=best_models[idx], messages=[
        {
            'role': 'system',
            'content': 'You are a world class cryptographer. You approach every problem with a fresh perspective and a keen eye for detail. You are the best of the best.',
        },
        {
            'role': 'user',
            'content': 'Decode this sentence: ' + sentences[0]
        }
    ])
    responses.append(response)
    print("-------------------------------")
    print(f"Model: {best_models[idx]}")
    print(responses[idx]['message']['content'])
    print("\n")


-------------------------------
Model: gemma3:12b
Alright, let's tackle this. My initial assessment suggests a Caesar cipher, a classic and often the first suspect in these kinds of exercises. The consistent shift in letters is a strong indicator. Let's try a shift of -1. This is a common starting point.

Applying a Caesar cipher with a shift of -1 (meaning each letter is replaced by the letter *before* it in the alphabet) yields:

"The place base effect is a solution in which people experience a improvement in health after taking a substance with no practical consequences because they believe it works."

**Therefore, the decoded sentence is: "The place base effect is a solution in which people experience a improvement in health after taking a substance with no practical consequences because they believe it works."**

**Reasoning & My Cryptographic Approach:**

1. **Initial Hypothesis:** The uniformity of the letter substitutions immediately pointed towards a substitution cipher. Caesa

DeepSeek jest nieco bardziej obiecujący.

In [45]:
response = chat(model='deepseek-r1:14b', messages=[
    {
        'role': 'system',
        'content': 'You are a world class cryptographer. Your speciality are Ceasar Ciphers. You are the best of the best.',
    },
    {
        'role': 'user',
        'content': 'Decode this sentence: ' + sentences[0]
    }
])
print("-------------------------------")
print(f"Model: deepseek-r1:14b")
print(response['message']['content'])
print("\n")


-------------------------------
Model: deepseek-r1:14b
<think>
Okay, so I have this message to decode using a Caesar cipher. The text is: "Uif qmbdfcp fggfdu jt b qifopnfopo jo xijdi qfpqmf fyqfsjfodf bo jnqspwfnfou jo ifbmui bgufs ubljoh b tvctubodf xjui op nfejdjobm qspqfsujft cfdbvtf uifz cfmjfwf ju xpslt." Alright, let's break this down.

First off, I know that a Caesar cipher shifts each letter in the message by a certain number of places in the alphabet. So, to decode it, I need to figure out what shift was used. The most common method is frequency analysis because letters like 'e', 't', and 'a' are more frequent in English.

Looking at the text, let me spot the most frequent words or letters. Hmm, "Uif" appears a few times. Maybe that's "the"? If I assume "Uif" is "the", then 'U' would be 'T', which means shifting back by 1 (since U is the next letter after T). So, shift = 1. Let me test this.

Taking the first word: "Uif". Shifting each letter back by 1:
- U -> T
- i -> h
- f -

Widzimy, że model poradził sobie z odgadnięciem szyfru (szyfr Cezara o przesunięciu jeden). Odpowiedź ta generowała się niestety bardzo bardzo długo. Wykonałem podobne testy korzystając z API i modelów nie postawionych lokalnie i z moich obserwacji wynika, że dopiero DeepSeek z 70B parametrów dawał całkowicie poprawne wyniki.

In [ ]:
response = chat(model='deepseek-r1:14b', messages=[
    {
        'role': 'system',
        'content': 'You are a world class cryptographer. Your speciality are Ceasar Ciphers. You are the best of the best.',
    },
    {
        'role': 'user',
        'content': 'Decode this sentence: ' + sentences[1]
    }
])
print("-------------------------------")
print(f"Model: deepseek-r1:14b")
print(response['message']['content'])
print("\n")


-------------------------------
Model: deepseek-r1:14b
<think>
Okay, so I need to decode this sentence: "Vjg jwocp dtckp ku pgwtqrncuvke, yjkej ogcpu kv ecp ejcpig cpf cfcrv vjtqwijqwv nkhg kp tgurqpug vq gzrgtkgpegu." Hmm, the user mentioned it's a Caesar Cipher, which I remember is a type of substitution cipher where each letter in the plaintext is shifted a certain number down or up the alphabet. The shift is usually consistent throughout the message.

First, let me think about how to approach this. Since it's a Caesar Cipher, each letter has been shifted by the same amount. My goal is to find out what that shift is so I can reverse it and get back the original message.

One common method for solving Caesar Ciphers is frequency analysis. In English, certain letters like 'E', 'T', 'A', etc., appear more frequently than others. Maybe I can look at the frequency of letters in the given ciphertext to guess which one corresponds to a common letter.

Looking at the sentence, I notice that

Ponownie, model wiedział jakie szyfr Cezara ma przesunięcie, ale ostatecznie nie rozszyfrował całego zdania.

In [ ]:
response = chat(model='deepseek-r1:14b', messages=[
    {
        'role': 'system',
        'content': 'You are a world class cryptographer. Your speciality are Ceasar Ciphers. You are the best of the best.',
    },
    {
        'role': 'user',
        'content': 'Decode this sentence: ' + sentences[2]
    }
])
print("-------------------------------")
print(f"Model: deepseek-r1:14b")
print(response['message']['content'])
print("\n")


-------------------------------
Model: deepseek-r1:14b
<think>
Okay, so I've got this message that's been encoded with a Caesar Cipher, and I need to decode it. I'm not super familiar with all the details of cryptography, but I know that Caesar Ciphers involve shifting each letter in the text by a certain number down the alphabet. For example, if you shift by 3, A becomes D, B becomes E, etc., and it wraps around so Z would become C.

The message given is: "Wkh Plojudp hashulphqw vkrzhg wkdw shrsoh duh zloolqj wr rehb dxwkrulwb hyhq zkhq wkhlu dfwlrqv pdb kdup rwkhuv."

First, I need to figure out by how many positions each letter has been shifted. There are 26 letters in the English alphabet, so shifting by any number between 1 and 25 would be possible. The challenge is determining which shift key is correct.

One common method for breaking Caesar Ciphers is frequency analysis. In English, certain letters like E, T, A, O, I, N, S, H, R, D, L, C, etc., appear more frequently than other

I tak to wyglądało dla każdego następnego zdania. Model potrafił przewidzieć dokładne przesunięcie, ale robił literówki i nie potrafił rozszyfrować całych zdań w 100% poprawnie.

## Podsumowanie

Podsumowując, nawet używając względnie niewielkiego modelu, byliśmy w stanie rozszyfrować zdania zakodowane szyfrem Cezara. Niestety ograniczenia sprzętowe nie pozwoliły na użycie większego DeepSeek'a, który poradził sobie ze wszystkim. Napiszmy teraz funkcję, która rozszyfruje zdania na inpucie, stosując przesunięcie i szyfr zaproponowany przez nasz model.

In [52]:
def decode_caesar_cipher(ciphertext: str, shift: int) -> str:
    """
    Decodes a Caesar cipher with the given shift.

    Parameters:
    ciphertext (str): The text to decode.
    shift (int): The number of positions each letter in the ciphertext was shifted.

    Returns:
    str: The decoded plaintext.
    """
    decoded_text = []

    for char in ciphertext:
        if char.isalpha():
            # Determine if the character is uppercase or lowercase
            ascii_offset = 65 if char.isupper() else 97
            # Shift the character back by the shift amount
            decoded_char = chr((ord(char) - ascii_offset - shift) % 26 + ascii_offset)
            decoded_text.append(decoded_char)
        else:
            # Non-alphabetic characters are added as is
            decoded_text.append(char)

    return ''.join(decoded_text)

decoded_sentences = []
for idx in range(len(sentences)):
    ciphertext = sentences[idx]
    shift = idx + 1
    decoded_text = decode_caesar_cipher(ciphertext, shift)
    print(decoded_text)
    decoded_sentences.append(decoded_text)


The placebo effect is a phenomenon in which people experience an improvement in health after taking a substance with no medicinal properties because they believe it works.
The human brain is neuroplastic, which means it can change and adapt throughout life in response to experiences.
The Milgram experiment showed that people are willing to obey authority even when their actions may harm others.
The bystander effect makes people less likely to offer help when more people are around—they assume that someone else will do it.
People remember the beginning and end of a list of information better, which is known as the primacy effect and the recency effect.
Prolonged stress can lead to health problems such as heart disease, a weakened immune system, and depression.
Emotions influence cognitive processes such as memory and decision-making—strong stress can hinder logical thinking.
Howard Gardner's theory of multiple intelligences suggests that there are various types of intelligence, such as 

In [ ]:
# Write the decoded sentences to a new file
output_file_path = 'decoded_sentences.txt'
with open(output_file_path, 'w') as file:
    for decoded_sentence in decoded_sentences:
        file.write(decoded_sentence + '\n')

print(f"Decoded sentences have been saved to {output_file_path}")


Decoded sentences have been saved to decoded_sentences.txt
